In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim 
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import gurobipy as gp
from gurobipy import GRB
from tqdm import tqdm 
from helper import *  

In [2]:
# set parameters  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
dataset = 'lsac'
sens_attribute = 'race'
nominal = False
merit = True
num_epochs = 30 
lr_theta = 1e-3 
lr_z = 1e-1 
epsilon = 1e-2 
delta = 0.5 

In [3]:
train_dataset = all_data(sens_attr=sens_attribute, train=True, dataset=dataset)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
X_train, y_train, sens_train = train_dataset.get_values()
flag = train_dataset.get_flag()
X_train_moments = train_dataset.get_moments()

X, y, s, idx = next(iter(train_loader))
num_feat = X.shape[1]

test_dataset = all_data(sens_attr=sens_attribute, train=False, dataset=dataset)
df = test_dataset.get_df(train=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)
X_test, y_test, sens_test = test_dataset.get_values()
X_test, y_test, sens_test = X_test.numpy(), y_test.numpy(), sens_test.numpy()
columns = list(df.columns)

In [4]:
# Define merit attributes 
dataset_features = train_dataset.get_columns()
dataset_features

['decile1b',
 'decile3',
 'lsat',
 'ugpa',
 'zfygpa',
 'zgpa',
 'fulltime',
 'fam_inc',
 'male',
 'tier',
 'race']

In [5]:
merit_attrs = [2,3]

In [6]:
# Define model 
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

In [7]:
X_train = torch.Tensor(X_train)
sens_train = torch.Tensor(list(sens_train))
y_train = torch.Tensor(list(y_train)) 
z0_train = torch.tensor([0 for i in range(len(y_train))])
z0_train = z0_train.to(device)
z0_train = proj_z_str(X_train_moments, merit_attrs, z0_train, X_train, y_train, sens_train, flag, epsilon=epsilon, delta=delta, merit=merit) 

model = LogisticRegressionModel(num_feat,1) 
model = model.to(device)  
opt = optim.Adam(model.parameters(), lr=lr_theta)

for _ in range(num_epochs): 
    train_err, train_loss, z = epoch_manual_str(train_loader, model, z0_train, X_train_moments, X_train, y_train, sens_train, flag, device, merit_attrs, delta=delta, epsilon=epsilon, lr_z=lr_z, opt=opt, merit=merit)
    z0_train = z 
    print(*("{:.6f}".format(i) for i in (train_err, train_loss)), sep="\t")

Academic license - for non-commercial use only - expires 2024-10-29
Using license file /Users/thodoris/gurobi.lic


100%|██████████| 227/227 [00:00<00:00, 650.29it/s]

Academic license - for non-commercial use only - expires 2024-10-29
Using license file /Users/thodoris/gurobi.lic



 34%|███▍      | 78/227 [00:00<00:00, 777.34it/s]

0.111607	0.502150


...

 67%|██████▋   | 152/227 [00:00<00:00, 747.00it/s]

0.097742	0.262100


100%|██████████| 227/227 [00:00<00:00, 754.44it/s]

Academic license - for non-commercial use only - expires 2024-10-29
Using license file /Users/thodoris/gurobi.lic


0.098565	0.268232


In [8]:
 # Evaluate Model
model.eval()
yp = get_preds(test_loader, model, device)
results_fairness, results_merit = metrics_str(merit_attrs, columns, X_test, y_test, sens_test, yp)

100%|██████████| 98/98 [00:00<00:00, 1101.09it/s]


In [9]:
results_fairness

,Acc_Total,SPD,EOD,DEO
0,0.8989,0.1003,0.0338,0.0852


In [10]:
results_merit

,lsat,ugpa
0,0.1044,0.1262
